In [ ]:
# Alp Sunol
# August 2019

# This script takes a molecule locations file and a reactions file 
# to compute stress and osmotic pressure.  Avoids downloading the 
# whole molecule position file at once.

# Import modules
import sys
import subprocess
import time
import os
import math
from numba import jit
import numpy as np
import pandas as pd

######################################################################################################################################
### Define Functions ###

## Function calculates osmotic pressure, stress, collision overlap and frequency data.
## We accumulate the numbers by taking as inputs the outputs of the previous timestep, then average outside the funciton if needed.
# In: name of dictionary of locations, start and end line numbers in reaction file, totals of relevant values
# Out: updated values
def calc_OP(pos_dic, start, end, stress_xx, stress_yy, stress_zz, stress_xy, stress_xz, stress_yz, stress_yx, stress_zx, stress_zy, \
            OP, OP_2, OP_ref, count_error, count_total, reflections, secondary_rxns, step_set, reac_dic, repeat, prev_col_inf, \
            av_overlap, av_overlap_rep):
    
    for i in range(start, end):
        step = int(round(reac_data[i][0]))
        step_set.add(step)
        id1 = int(round(reac_data[i][4]))
        id2 = int(round(reac_data[i][5]))
        
        del_pos1 = pos_dic[(step+2,id1)][1:4] - pos_dic[(step+1,id1)][1:4]
        del_pos2 = pos_dic[(step+2,id2)][1:4] - pos_dic[(step+1,id2)][1:4] 
        del_1_amount = np.linalg.norm(del_pos1)
        del_2_amount = np.linalg.norm(del_pos1)
        
        dist_init = pos_dic[(step+1,id1)][1:4] - pos_dic[(step+1,id2)][1:4]
        overlap = 2 - np.linalg.norm(dist_init)
        dist_fin = pos_dic[(step+2,id1)][1:4] - pos_dic[(step+2,id2)][1:4]
        
        if np.linalg.norm(del_pos1)>1e-10:
            
            count_total += 1
            av_overlap += 2*np.linalg.norm(del_pos1)
            
            if prev_col_inf[id1-1] == id2 and prev_col_inf[id2-1] == id1:
                repeat += 1
                av_overlap_rep += 2*np.linalg.norm(del_pos1)
                
            prev_col_inf[id1-1] = id2
            prev_col_inf[id2-1] = id1
            
            if abs(overlap) < 1 and del_1_amount < 1 and del_2_amount < 1 \
                and np.linalg.norm(dist_fin) < 3 and np.linalg.norm(dist_init) < 3:
                OP += np.dot(dist_fin,del_pos1)

                if step%10 >= 1: # loop to count number of secondary reactions
                    if step%10 == 1 and (step - 1) in step_set:
                        secondary_rxns += 1 
                        OP_2 += np.dot(dist_fin,del_pos1)

                    elif step%10 > 1: 
                        secondary_rxns += 1
                        OP_2 += np.dot(dist_fin,del_pos1)

                stress_xx[id1-1] -= dist_fin[0]*del_pos1[0]
                stress_yy[id1-1] -= dist_fin[1]*del_pos1[1]
                stress_zz[id1-1] -= dist_fin[2]*del_pos1[2]
                stress_xy[id1-1] -= dist_fin[0]*del_pos1[1]
                stress_xz[id1-1] -= dist_fin[0]*del_pos1[2]
                stress_yz[id1-1] -= dist_fin[1]*del_pos1[2]
                stress_yx[id1-1] -= dist_fin[1]*del_pos1[0]
                stress_zx[id1-1] -= dist_fin[2]*del_pos1[0]
                stress_zy[id1-1] -= dist_fin[2]*del_pos1[1]

                stress_xx[id2-1] -= dist_fin[0]*del_pos1[0]
                stress_yy[id2-1] -= dist_fin[1]*del_pos1[1]
                stress_zz[id2-1] -= dist_fin[2]*del_pos1[2]
                stress_xy[id2-1] -= dist_fin[0]*del_pos1[1]
                stress_xz[id2-1] -= dist_fin[0]*del_pos1[2]
                stress_yz[id2-1] -= dist_fin[1]*del_pos1[2]
                stress_yx[id2-1] -= dist_fin[1]*del_pos1[0]
                stress_zx[id2-1] -= dist_fin[2]*del_pos1[0]
                stress_zy[id2-1] -= dist_fin[2]*del_pos1[1]

            else:
                reflections +=1 
                for i in range(3): # find out which dimension the reflection is
                    if abs(del_pos1[i])>2:
                        del_pos1[i] -= np.sign(del_pos1[i])*box
                        dist_fin[i] += np.sign(del_pos1[i])*box

                    if abs(del_pos2[i])>2:
                        del_pos2[i] += np.sign(del_pos1[i])*box
                        dist_fin[i] += box

                    elif abs(dist_fin[i]) > 2:
                        ref_dir = i
                        if del_pos1[i] > 0:
                            dist_fin[i] += box
                        else:
                            dist_fin[i] -= box

                OP_ref += np.dot(dist_fin,del_pos1)
                    
            
            
    return stress_xx, stress_yy, stress_zz, stress_xy, stress_xz, stress_yz, stress_yx, stress_zx, stress_zy, \
    OP, OP_2, OP_ref, count_error, count_total, reflections, secondary_rxns, step_set, reac_dic, repeat, prev_col_inf,\
    av_overlap, av_overlap_rep;


## Function creates dictionary of locations before and after collisions:
# In: name of position document, start line number, end line number
# Out: dictionary with format --> pos_dic(timestep,ID) = [type,pos_x, pos_y, pos_z]
def create_dic(name_pos, start, end):
    #pos_data = pd.read_csv(name_pos, delimiter=' ', header=None, skiprows = N*(start), nrows=N*(end-start))
    #len_pos = int(pos_data.shape[0])
    #print(len_pos)
    #pos_dic = dict()
    
   # for j in range(len_pos):
     #   tup_in = (pos_data[0][j].astype(int),pos_data[6][j].astype(int))
     #   array_out = np.array([pos_data[1][j].astype(int),pos_data[3][j].astype(float)/rad,pos_data[4][j].astype(float)/rad,\
        # pos_data[5][j].astype(float)/rad])
     #   pos_dic[tup_in] = array_out
     #   if j%100000 ==0:
      #      print(j)
            
    pos_data = np.loadtxt(name_pos, delimiter=' ', skiprows = start, max_rows=end-start)
    len_pos = int(pos_data.shape[0])
    pos_dic = dict()
    
    for j in range(len_pos):
        tup_in = (pos_data[j][0].astype(int),pos_data[j][6].astype(int))
        array_out = np.array([pos_data[j][1].astype(int),pos_data[j][3].astype(float)/rad,pos_data[j][4].astype(float)/rad,\
                              pos_data[j][5].astype(float)/rad])
        pos_dic[tup_in] = array_out
            
    return pos_dic


######################################################################################################################################

t0 = time.time() # start the timer


## Enter in key parameters (for non-dimensionalizing and processing/averaging)
# Dimensionless numbers non-dimensionalized by tRNA
N = 42 # number of particles
del_t = 1e-4 # time step in s (or dimensionless)
D = 1 # um^2/s (or dimensionless)
rad = 1 # radius of particle in um (or dimensionless)
box = 11.47457627
dt = del_t*D/(rad**2) # unitless time


# Input and output files
name_pos = '/Users/Akshay/Downloads/expt-3-Molpos-mono_low.txt' # file with positions of all molecules at all times
name_reac = '/Users/Akshay/Downloads/expt-3-Reactions-mono_low.txt' # collisions/reactions file
name_out = 'output.txt' # name of output

# Import reaction file:
reac_data = np.genfromtxt(name_reac, delimiter=' ', usecols = (0,2,3,4,5,6,7,8))
len_reac = int(reac_data.shape[0]) # length of file


for i in range(len(reac_data)):
    reac_data[i][0] /= del_t # convert time to time step
    reac_data[i][1:3] /= rad # normalize size 

# Find start_point which is after equilibration period:    
for i in range(len_reac):
    number = 1.020/del_t 
    if reac_data[i][0] >= number:
        start_point = i
        break

end_point = int(len(reac_data)) # end of reaction file
num_timesteps = math.ceil((reac_data[end_point-int(1)][0]-reac_data[start_point][0])/10) # to normalize later, this is time spent diffusing

## Initialization of parameters to calculate
# Initialize stress components
stress_xx = np.zeros(N)
stress_yy = np.zeros(N)
stress_zz = np.zeros(N)
stress_xy = np.zeros(N)
stress_xz = np.zeros(N)
stress_yz = np.zeros(N)
stress_yx = np.zeros(N)
stress_zx = np.zeros(N)
stress_zy = np.zeros(N)

OP = 0 # initialize OP
OP_2 = 0
OP_ref = 0
count_error = 0
count_total = 0
reflections = 0
secondary_rxns = 0
step_set = set()
reac_dic = dict()

# Reaction collision stuff:
repeat = 0
prev_col_inf = np.zeros(N)
av_overlap = 0
av_overlap_rep = 0

## Set up dividing the downloading of the molecule file:
num_div = .01/dt # number of blocks to divide data
space = (end_point - start_point)/num_div # space to divide reaction file

splice_indicies = np.array([]) # indicies to splice between in reaction file
for i in range(start_point,end_point,int(space)):
    splice_indicies = np.append(splice_indicies,i)
splice_indicies = np.append(splice_indicies,int(end_point))


## Loop through these indicies to download data in incremements:
for i in range(len(splice_indicies)-1):
    
    # Start and end points in reaction file:
    start_idx = int(splice_indicies[i])
    end_idx = int(splice_indicies[i+1]-1)
    
    # What this corresponds to in position file, adding a +-3 timestep buffer:
    start_dict = N*(int(round(reac_data[start_idx][0]))-3)
    end_dict = N*(int(round(reac_data[end_idx][0]))+3)
    
    # Create dictionary with relevant section of data:
    pos_dic = dict() # start with an empty dictionary 
    pos_dic = create_dic(name_pos, start_dict, end_dict)
    
    # Tally up osmotic pressure from this window:
    stress_xx, stress_yy, stress_zz, stress_xy, stress_xz, stress_yz, stress_yx, stress_zx, stress_zy, \
    OP, OP_2, OP_ref, count_error, count_total, reflections, secondary_rxns, step_set, reac_dic, \
    repeat, prev_col_col_inf, av_overlap, av_overlap_rep = \
    calc_OP(pos_dic, start_idx, end_idx, stress_xx, stress_yy, stress_zz, stress_xy, stress_xz, stress_yz, stress_yx, stress_zx, stress_zy, \
    OP, OP_2, OP_ref, count_error, count_total, reflections, secondary_rxns, step_set, reac_dic, repeat, prev_col_inf, \
    av_overlap, av_overlap_rep)
    
    pos_dic.clear() # clear the dictionary to save space
    
    print('done with round of OP: round ', i)

    

# Average the stress components.
stress_av_xx = np.sum(stress_xx)/(2*dt*N)/num_timesteps
stress_av_yy = np.sum(stress_yy)/(2*dt*N)/num_timesteps
stress_av_zz = np.sum(stress_zz)/(2*dt*N)/num_timesteps
stress_av_xy = np.sum(stress_xy)/(2*dt*N)/num_timesteps
stress_av_xz = np.sum(stress_xz)/(2*dt*N)/num_timesteps
stress_av_yz = np.sum(stress_yz)/(2*dt*N)/num_timesteps
stress_av_yx = np.sum(stress_yx)/(2*dt*N)/num_timesteps
stress_av_zx = np.sum(stress_zx)/(2*dt*N)/num_timesteps
stress_av_zy = np.sum(stress_zy)/(2*dt*N)/num_timesteps

## Write everything onto text file:
out = open(name_out, "w")

out.write('Components of stress:'+ '\n')
out.write(str(stress_av_xx)+ '\n')
out.write(str(stress_av_yy)+ '\n')
out.write(str(stress_av_zz)+ '\n')
out.write(str(stress_av_xy)+ '\n')
out.write(str(stress_av_xz)+ '\n')
out.write(str(stress_av_yz)+ '\n')
out.write(str(stress_av_yx)+ '\n')
out.write(str(stress_av_zx)+ '\n')
out.write(str(stress_av_zy)+ '\n')
out.write('')


OP_scaled = OP/(3*N*dt)/num_timesteps
OP_2_scaled = OP_2/(3*N*dt)/num_timesteps
OP_ref_scaled = OP_ref/(3*N*dt)/num_timesteps
OP_adjusted = OP_scaled*count_total/(count_total-reflections)

# OP by equation of state
phi = N*(4/3)*math.pi/box**3 # volume fraction
g_contact = (1-0.5*phi)/(1-phi)**3
D_long = D/(1+2*phi*g_contact)
carnahan_OP = 4*phi*g_contact

string = 'phi = '+ str(phi) + '\n'
out.write(string)
string = 'OP_raw = ' + str(OP_scaled) + '\n'
out.write(string)
string = 'OP_ad_hoc_scale = ' + str(OP_adjusted) + '\n'
out.write(string)
string = 'OP_sim = '+ str(OP_scaled+OP_ref_scaled) + '\n'
out.write(string)
string = 'OP_secondary = '+ str(OP_2_scaled) +str( 100*(OP_2_scaled/(OP_scaled+OP_ref_scaled))) + '\n'
out.write(string)
string = 'OP_CSEOS = '+ str(carnahan_OP) + '\n'
out.write(string)
string = 'Long time diffusivity = '+ str(D_long) + '\n'
out.write(string)
string = 'Total collisions = '+ str(count_total) + '\n'
out.write(string)
string = 'Repeat collisions = ' + str(repeat) + '\n'
out.write(string)
string = 'Average overlap = ' + str(av_overlap/count_total) + '\n'
out.write(string)
string = 'Repeat overlap = ' + str(av_overlap_rep/repeat) + '\n'
out.write(string)
#out.write(count_total, repeat, av_overlap/count_total, av_overlap_rep/repeat)
out.close()

t1 = time.time()
print('total time elapsed = ', t1-t0)

done with round of OP: round  0
done with round of OP: round  1
done with round of OP: round  2
done with round of OP: round  3
done with round of OP: round  4
done with round of OP: round  5
done with round of OP: round  6
done with round of OP: round  7
done with round of OP: round  8
done with round of OP: round  9
done with round of OP: round  10
done with round of OP: round  11
done with round of OP: round  12
done with round of OP: round  13
done with round of OP: round  14
done with round of OP: round  15
done with round of OP: round  16
done with round of OP: round  17
done with round of OP: round  18
done with round of OP: round  19
done with round of OP: round  20
done with round of OP: round  21
done with round of OP: round  22
done with round of OP: round  23
done with round of OP: round  24
done with round of OP: round  25
done with round of OP: round  26
done with round of OP: round  27
done with round of OP: round  28
done with round of OP: round  29
done with round of O

In [175]:
reflections

18379

In [176]:
total

NameError: name 'total' is not defined

In [177]:
total


NameError: name 'total' is not defined

In [178]:
count_total

16947